# 1. Import Dependencies

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os

## Dataset
This specific dataset seperates hand gestures into 4 different classes.

- thumbsup
- thumbsdown
- thankyou
- livelong

The information about each hand gesture is contained into 42 features.

In [2]:
CSV_COLUMN_NAMES = np.arange(43)
CSV_COLUMN_NAMES = [str(item) for item in CSV_COLUMN_NAMES]
CSV_COLUMN_NAMES[42] = "Species"
SPECIES = ['stop', 'onefingerup', 'twofingerup', 'thumbsup']
CSV_PATH = os.path.join("src", "dataHandGesture", "file_0.csv")

In [3]:
df = pd.read_csv(CSV_PATH, names=CSV_COLUMN_NAMES, header=0)
train=df.sample(frac=0.8,random_state=200) #random state is a seed value
test=df.drop(train.index)

In [4]:
train.head()

,0,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,39,40,41,Species
35,85,43,69,-8,39,-59,25,-99,31,-132,...,33,19,62,-30,55,-16,54,4,55,3
37,54,33,54,-6,39,-35,24,-59,18,-77,...,18,-16,44,-17,36,-5,34,3,35,3
22,23,71,-3,66,-21,40,-8,18,12,5,...,38,41,18,34,11,25,32,21,48,2
2,19,104,-21,100,-51,71,-68,45,-87,25,...,-71,53,42,64,7,68,-17,72,-39,0
21,7,71,-9,65,-16,45,1,26,23,12,...,21,17,19,24,16,24,32,25,43,2


In [5]:
train_y = train.pop('Species')
test_y = test.pop('Species')
train.head() # the species column is now gone

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
35,85,43,69,-8,39,-59,25,-99,31,-132,...,-5,33,19,62,-30,55,-16,54,4,55
37,54,33,54,-6,39,-35,24,-59,18,-77,...,0,18,-16,44,-17,36,-5,34,3,35
22,23,71,-3,66,-21,40,-8,18,12,5,...,8,38,41,18,34,11,25,32,21,48
2,19,104,-21,100,-51,71,-68,45,-87,25,...,29,-71,53,42,64,7,68,-17,72,-39
21,7,71,-9,65,-16,45,1,26,23,12,...,22,21,17,19,24,16,24,32,25,43


In [6]:
train.shape  # we have 120 entires with 4 features

(31, 42)

## Input Function
Remember that nasty input function we created earlier. Well we need to make another one here! Fortunatly for us this one is a little easier to digest.

In [7]:
def input_fn(features, labels, training=True, batch_size=256):
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle and repeat if you are in training mode.
    if training:
        dataset = dataset.shuffle(1000).repeat()
    
    return dataset.batch(batch_size)

In [8]:
# Feature columns describe how to use the input.
my_feature_columns = []
for key in train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))
print(my_feature_columns)

[NumericColumn(key='0', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='1', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='2', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='3', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='4', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='5', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='6', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='7', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='8', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='9', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='10', shape=(1,), default_value=None, dty

# 2. Building the Model
And now we are ready to choose a model. For classification tasks there are variety of different estimators/models that we can pick from. Some options are listed below.

- DNNClassifier (Deep Neural Network)
- LinearClassifier

We can choose either model but the DNN seems to be the best choice. This is because we may not be able to find a linear coorespondence in our data.

In [9]:
MODEL_PATH = os.path.join("Tensorflow", "workspace", "models", "my_hand_gesture_model")

In [10]:
# Build a DNN with 2 hidden layers with 30 and 10 hidden nodes each.
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 30 and 10 nodes respectively.
    hidden_units=[30, 10],
    # The model must choose between 3 classes.
    n_classes=len(SPECIES),
    model_dir= MODEL_PATH)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'Tensorflow\\workspace\\models\\my_hand_gesture_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


## Training

In [11]:
classifier.train(
    input_fn=lambda: input_fn(train, train_y, training=True),
    steps=5000)
# We include a lambda to avoid creating an inner function previously

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into Tensorflow\workspace\models\my_hand_gesture_model\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 14.537058, step = 0
INFO:tensorflow:global_step/sec: 85.9796
INFO:tensorflow:loss = 0.5697803, step = 100 (1.159 sec)
INFO:tensorflow:global_step/sec: 129.694
INFO:tensorflow:loss = 0.39779305, step = 200 (0.771 se

## Export Model

In [12]:
# To export
feature_spec = tf.feature_column.make_parse_example_spec(my_feature_columns);
export_input_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(feature_spec);
export_path = classifier.export_saved_model(MODEL_PATH, export_input_fn, as_text=True);

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Signatures INCLUDED in export for Classify: ['serving_default', 'classification']
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['predict']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:Restoring parameters from Tensorflow\workspace\models\my_hand_gesture_model\model.ckpt-5000
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: Tensorflow\workspace\models\my_hand_gesture_model\temp-1636134306\saved_model.pbtxt


## Load Model

In [13]:
# Loading the estimator
predict_fn = tf.saved_model.load(export_path).signatures['predict']
predict_fn

<ConcreteFunction pruned(examples) at 0x3A4E2910>

In [14]:
# Convert input data into serialized Example strings.
examples = []
for index, row in test.iterrows():
    feature = {}
    for col, value in row.iteritems():
        feature[col] = tf.train.Feature(float_list=tf.train.FloatList(value=[value]))
    example = tf.train.Example(
        features=tf.train.Features(
            feature=feature
        )
    )
    examples.append(example.SerializeToString())

# Convert from list to tensor
examples = tf.constant(examples)

print(test)
print(feature)

     0    1   2   3   4   5   6   7   8   9  ...  32  33  34  35  36  37  38  \
4    3  108 -29  76 -48  44 -57  19 -68   2  ...  32 -48  28  35  42  13  50   
12   6   72 -22  52 -40  28 -29  12  -9   8  ...   6  34  41  -3  41   4  32   
14   9   59 -15  41 -35  17 -35   0 -18  -1  ...   3  28  41   3  21  12  14   
15   2   42 -19  36 -38  20 -34  11 -15  11  ...   7  38  39   0  26  18  19   
16  11   53 -14  50 -39  27 -40   9 -18   7  ...   9  33  48   0  28  13  21   
25  17   63  -3  60 -18  41  -7  23   9  11  ...  10  32  34  16  31  10  23   
26  27   77   0  65 -18  41 -10  19   4   5  ...  10  29  36  15  32   8  25   
27  32   77   3  67 -15  45 -11  21   3   7  ...   8  13  47  10  37   3  32   

    39  40  41  
4   -2  57 -18  
12  22  23  33  
14  27  16  33  
15  34  19  42  
16  30  23  37  
25  27  18  41  
26  25  23  39  
27  24  28  42  

[8 rows x 42 columns]
{'0': float_list {
  value: 32.0
}
, '1': float_list {
  value: 77.0
}
, '2': float_list {
  value: 3.0

In [15]:
# make predictions of all testset
predictions = predict_fn(examples=examples)
print(predictions)

{'probabilities': <tf.Tensor: shape=(8, 4), dtype=float32, numpy=
array([[1.4236303e-01, 2.9181275e-01, 3.6879134e-01, 1.9703291e-01],
       [8.3539278e-05, 9.9334586e-01, 6.5706577e-03, 1.2142205e-08],
       [6.2621336e-07, 9.9922240e-01, 7.6804298e-04, 8.9469186e-06],
       [3.9488223e-06, 9.9226111e-01, 7.7348296e-03, 1.5686528e-07],
       [8.8279796e-07, 9.9954128e-01, 4.4942606e-04, 8.4294361e-06],
       [1.0929435e-09, 1.9701957e-03, 9.9802822e-01, 1.5716212e-06],
       [2.8866568e-13, 2.9588876e-02, 9.6995944e-01, 4.5170600e-04],
       [7.6922445e-16, 4.1139293e-02, 9.5624590e-01, 2.6148378e-03]],
      dtype=float32)>, 'all_class_ids': <tf.Tensor: shape=(8, 4), dtype=int32, numpy=
array([[0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3]])>, 'classes': <tf.Tensor: shape=(8, 1), dtype=string, numpy=
array([[b'2'],
       [b'1'],
       [b'1'],
       [b'1'],
      

In [16]:
# print results
for idx, resultPred in enumerate(predictions["class_ids"]):
    class_id = resultPred[0]
    probability = predictions['probabilities'][idx][class_id]
    print(probability)
    print(f"\tPrediction is {SPECIES[class_id]} {100 * probability :.2f}%")
    print(f"\tExpected: {SPECIES[test_y.iloc[idx]]}")

tf.Tensor(0.36879134, shape=(), dtype=float32)
	Prediction is twofingerup 36.88%
	Expected: stop
tf.Tensor(0.99334586, shape=(), dtype=float32)
	Prediction is onefingerup 99.33%
	Expected: onefingerup
tf.Tensor(0.9992224, shape=(), dtype=float32)
	Prediction is onefingerup 99.92%
	Expected: onefingerup
tf.Tensor(0.9922611, shape=(), dtype=float32)
	Prediction is onefingerup 99.23%
	Expected: onefingerup
tf.Tensor(0.9995413, shape=(), dtype=float32)
	Prediction is onefingerup 99.95%
	Expected: onefingerup
tf.Tensor(0.9980282, shape=(), dtype=float32)
	Prediction is twofingerup 99.80%
	Expected: twofingerup
tf.Tensor(0.96995944, shape=(), dtype=float32)
	Prediction is twofingerup 97.00%
	Expected: twofingerup
tf.Tensor(0.9562459, shape=(), dtype=float32)
	Prediction is twofingerup 95.62%
	Expected: twofingerup


## Evaluation

In [17]:
eval_result = classifier.evaluate(
    input_fn=lambda: input_fn(test, test_y, training=False))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-11-05T18:45:11
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from Tensorflow\workspace\models\my_hand_gesture_model\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.28902s
INFO:tensorflow:Finished evaluation at 2021-11-05-18:45:11
INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.875, average_loss = 0.25528398, global_step = 5000, loss = 0.25528398
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: Tensorflow\workspace\models\my_hand_gesture_model\model.ckpt-5000

Test set accuracy: 0.875



## Predictions
Now that we have a trained model it's time to use it to make predictions. I've written a little script below that allows you to type the features of a flower and see a prediction for its class.

In [18]:
def input_fn(features, batch_size=256):
    # Convert the inputs to a Dataset without labels.
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

features = CSV_COLUMN_NAMES[:-1]
predict = {}

for rowNum in range(5):
    expected = SPECIES[test_y.iloc[rowNum]]
    for idx, feature in enumerate(features):
      predict[feature] = [test.iloc[rowNum][idx]] #to predict the first row of test

    predictions = classifier.predict(input_fn=lambda: input_fn(predict))
    for pred_dict in predictions:
        class_id = pred_dict['class_ids'][0]
        probability = pred_dict['probabilities'][class_id]

        print("\n")
        print('Prediction is "{}" ({:.1f}%)'.format(
            SPECIES[class_id], 100 * probability))
        print(f"Expected is {expected}")

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from Tensorflow\workspace\models\my_hand_gesture_model\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


Prediction is "twofingerup" (36.9%)
Expected is stop
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from Tensorflow\workspace\models\my_hand_gesture_model\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


Prediction is "onefingerup" (99.3%)
Expected is onefingerup
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from Tensorflow\workspace\models\my_hand_gesture_model\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done run